By: Alvaro Loya Villalpando (aloyavil@caltech.edu)

Last revised: September 15, 2020

Note: To rsync data: rsync -avzhe ssh dmkidaq@131.215.193.26:/home/dmkidaq/usrp_data/200828 usrp_data

In [1]:
import usrp_correlated_noise_removal as ucnr
import imp #use imp.reload(ucnr) to reload module after editing

### various datasets (choose one)

In [2]:
data = 'USRP_Noise_20200828_114833.h5'

#----- 8 tone data: bandwidth of USRP + resonance tones data-------

# 'USRP_Noise_20200828_114833.h5' #res tones new - get multiple blobs for a single tone???

# 'USRP_Noise_20200828_115521.h5' #3060_3095  [0-7]

# 'USRP_Noise_20200828_115646.h5', #3020_3055 [8-15]

# 'USRP_Noise_20200828_115132.h5' #3105_3140  [16-23]

# 'USRP_Noise_20200828_115319.h5' #3145-3180  [24-31] 

#----- 2 tone data-------
#'USRP_Noise_20200903_150511.h5' #minor beating, no shark tooth

#'USRP_Noise_20200903_151311.h5' #major beating, sharp shark tooth

### load data

In [3]:
data_freqs, data_noise, time_correction_undecimated = ucnr.unavg_noi(data,verbose=True)



Data taken 20200828_114833
Reported LO is 3100.0 MHz
Reported rate is 200.000000 MHz
Reported decimation is 40
	Effective rate is 5000.000000 kHz
Reported amplitudes are [0.11885022 0.11885022 0.11885022 0.11885022 0.11885022 0.11885022
 0.11885022 0.11885022]
	Powers are [-29.5 -29.5 -29.5 -29.5 -29.5 -29.5 -29.5 -29.5] dBm
Tones are [3134. 3139. 3143. 3148. 3153. 3160. 3165. 3169.] MHz


### select a subset of the data (if you want)

In [4]:
data_noise = ucnr.reduced_data(data_noise,start=0,end=len(data_noise))

### calculate amplitude and arc length

In [5]:
coh_data_1, coh_data_2 = ucnr.calc_mag_phase(data_noise)

### further decimate amplitude and arc length data

In [6]:
%%time
#for no further decimation, set dec to 1
dec = 500 #set secimation to 500
coh_data_1_decimated, coh_data_2_decimated, time_correction_decimated= ucnr.further_decimation(decimation=dec,coh_data_1=coh_data_1,coh_data_2=coh_data_2,
                                                               time_correction_undecimated=time_correction_undecimated,plot=True,data=data)

Creation of the directory /Users/alvaro/Desktop/Caltech/group/analysis/usrp_noise_correlation/dec_vs_undec failed, it may already exist
CPU times: user 24.3 s, sys: 3.86 s, total: 28.1 s
Wall time: 28.4 s


### compute frequency spectrum

In [7]:
J_freqs = ucnr.compute_spectrum(coh_data_1,coh_data_1_decimated,time_correction_undecimated,time_correction_decimated,decimated_spectrum=False,avg=33)

### noise removal

In [8]:
%%time
data_freqs,coeff1,coeff2,coh_data_1_dirty,coh_data_2_dirty,coh_data_1_clean,coh_data_2_clean=ucnr.noise_removal(coh_data_1=coh_data_1,coh_data_2=coh_data_2, coh_data_1_decimated=coh_data_1_decimated,
                  coh_data_2_decimated=coh_data_2_decimated,decimation=dec,data_freqs=data_freqs,decimated_data=False,ommitted_vals=[],plot=False,data=data)

working on tone 0, time is 2020-09-15 14:58:36.530761
working on magnitude 2020-09-15 14:58:36.530889
working on phase 2020-09-15 14:58:37.232992
working on tone 1, time is 2020-09-15 14:58:38.544453
working on magnitude 2020-09-15 14:58:38.544572
working on phase 2020-09-15 14:58:39.383701
working on tone 2, time is 2020-09-15 14:58:40.097245
working on magnitude 2020-09-15 14:58:40.097412
working on phase 2020-09-15 14:58:40.771111
working on tone 3, time is 2020-09-15 14:58:41.446034
working on magnitude 2020-09-15 14:58:41.446193
working on phase 2020-09-15 14:58:42.103169
working on tone 4, time is 2020-09-15 14:58:42.753214
working on magnitude 2020-09-15 14:58:42.753376
working on phase 2020-09-15 14:58:43.451001
working on tone 5, time is 2020-09-15 14:58:44.115497
working on magnitude 2020-09-15 14:58:44.115661
working on phase 2020-09-15 14:58:44.758976
working on tone 6, time is 2020-09-15 14:58:45.382767
working on magnitude 2020-09-15 14:58:45.382934
working on phase 2020-

### calculate PSDs

In [9]:
psd_amp_dirty, psd_phase_dirty = ucnr.psd_calc(data_freqs=data_freqs,coh_data_1=coh_data_1_dirty,coh_data_2=coh_data_2_dirty,time_correction=time_correction_undecimated,avg_num=33,print_progress=False)

In [10]:
psd_amp_clean, psd_phase_clean = ucnr.psd_calc(data_freqs=data_freqs,coh_data_1=coh_data_1_clean,coh_data_2=coh_data_2_clean,time_correction=time_correction_undecimated,avg_num=33,print_progress=False)

In [11]:
ucnr.psd_plot(data_freqs=data_freqs,J_freqs=J_freqs,PSD_avg_1=psd_amp_dirty,PSD_avg_2=psd_phase_dirty,PSD_avg_1_clean=psd_amp_clean,PSD_avg_2_clean=psd_phase_clean,data=data)

### restore clean data back to raw timestreams and add to h5 file

In [28]:
ucnr.save_clean_timestreams(h5_file=data,data_raw=data_noise,cd1=coh_data_1_dirty,cd2=coh_data_2_dirty,cd1_clean=coh_data_1_clean,cd2_clean=coh_data_2_clean,coeff1=coeff1,coeff2=coeff2,override=True)

cleaned_data already exists! If you set override=False, nothing will happen.
saving clean_data to USRP_Noise_20200828_114833.h5!
